# Emotion Recognition Using Convoluted Neural Network

By: Aurelius Justin Lim

This notebook aims to create a CNN in order to recognition human emotions. The emotions to be experimented are anger, happiness, and sadness. To do this, the network is trained from a dataset taken from kaggle. A training, test and validation set is used to ensure the correctness and optimal performance of the model. Ultimately, the model created in the notebook is integrated into an application that allows users to upload a photo and allow the model to determine the illustrated emotion. 

**Human Face Emotions By SANIDHYAK**

Reference: https://www.kaggle.com/datasets/sanidhyak/human-face-emotions?resource=download

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


base_dir = 'data'
classes = ['Sad', 'Angry', 'Happy']  

# Set paths for the train, validation, and test directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create directories
create_dir(train_dir)
create_dir(val_dir)
create_dir(test_dir)
for cls in classes:
    create_dir(os.path.join(train_dir, cls))
    create_dir(os.path.join(val_dir, cls))
    create_dir(os.path.join(test_dir, cls))

# Split data and move files
for cls in classes:
    # Source directory for each class
    src_dir = os.path.join(base_dir, cls)
    
    # Get a list of all files in the source directory
    files = [os.path.join(src_dir, f) for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    
    # Split data: 80% train, 10% validation, 10% test
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)  # First, split into 80% train and 20% test
    val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)  # Split the 20% into two parts: 10% val, 10% test
    
    # Function to copy files to a target directory
    def copy_files(files, target_dir):
        for file in files:
            shutil.copy(file, os.path.join(target_dir, cls))
    
    # Copy files to the respective directories
    copy_files(train_files, train_dir)
    copy_files(val_files, val_dir)
    copy_files(test_files, test_dir)

print("Data successfully split into train, validation, and test sets.")


Data successfully split into train, validation, and test sets.


In [3]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 211 images belonging to 3 classes.
Found 26 images belonging to 3 classes.
Found 33 images belonging to 3 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/25
6/6 [==============================] - 14s 2s/step - loss: 1.8833 - accuracy: 0.3073
Epoch 2/25
6/6 [==============================] - 9s 1s/step - loss: 1.0803 - accuracy: 0.4078
Epoch 3/25
6/6 [==============================] - 11s 1s/step - loss: 1.0851 - accuracy: 0.3520
Epoch 4/25
6/6 [==============================] - 13s 2s/step - loss: 1.0575 - accuracy: 0.3911
Epoch 5/25
6/6 [==============================] - 15s 3s/step - loss: 1.0044 - accuracy: 0.5052
Epoch 6/25
6/6 [==============================] - 13s 2s/step - loss: 0.9881 - accuracy: 0.4190
Epoch 7/25
6/6 [==============================] - 13s 2s/step - loss: 1.0353 - accuracy: 0.4860
Epoch 8/25
6/6 [==============================] - 10s 1s/step - loss: 0.9654 - accuracy: 0.5698
Epoch 9/25
6/6 [==============================] - 11s 2s/step - loss: 0.9154 - accuracy: 0.5475
Epoch 10/25
6/6 [==============================] - 12s 2s/step - loss: 0.9396 - accuracy: 0.5312
Epoch 11/25
6/6 [=======================

In [6]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

1/1 [==============================] - 1s 663ms/step - loss: 1.1621 - accuracy: 0.5625
Test accuracy: 0.5625
